# Medical Chatbot Inference & Interface (using checkpoint-2000)

This notebook demonstrates how to load your current best checkpoint and interact with your domain-specific chatbot. When you finish retraining, you can simply update the checkpoint path to use your best model.

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import os
import torch

# Use the correct local checkpoint directory for tokenizer and adapter
ckpt_dir = os.path.abspath("./checkpoint-2000")

# Load tokenizer from local checkpoint directory
tokenizer = AutoTokenizer.from_pretrained(
    ckpt_dir,
    local_files_only=True
)

# Load base model (quantized, force CPU to avoid OOM)
bnb_config = BitsAndBytesConfig(load_in_4bit=True, llm_int8_enable_fp32_cpu_offload=True)
base_model = AutoModelForCausalLM.from_pretrained(
    "Featherless-Chat-Models/Mistral-7B-Instruct-v0.2",
    device_map="cpu",  # Force CPU to avoid GPU OOM
    quantization_config=bnb_config,
    local_files_only=True
)

# Load LoRA adapter from local checkpoint directory
model = PeftModel.from_pretrained(
    base_model,
    ckpt_dir,
    local_files_only=True
)

# # Move model to GPU if available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = model.to(device)

Loading weights: 100%|██████████| 291/291 [03:27<00:00,  1.40it/s, Materializing param=model.norm.weight]                              



## Try the chatbot: Single prompt example

In [ ]:
# Set device for inference
device = "cuda" if torch.cuda.is_available() else "cpu"

prompt = "Instruction: What are the symptoms of diabetes? Question:  Response:"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

NameError: name 'device' is not defined

## Interactive Chatbot Interface (Gradio)
You can use this cell to launch a simple web interface for your chatbot. When you retrain and have a better checkpoint, just update `ckpt_dir` above.

In [ ]:
import gradio as gr

def chat(query):
    inputs = tokenizer(query, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

gr.Interface(
    fn=chat,
    inputs=gr.Textbox(lines=2, label="Your medical question"),
    outputs=gr.Textbox(label="Chatbot Response"),
    title="Medical Chatbot",
    description="Ask a medical question and get a response from your fine-tuned model (checkpoint-2000)."
).launch()

---
**When you finish retraining:**
- Change `ckpt_dir` to your best checkpoint directory (e.g., `./checkpoint-8000` or `./best_model`)
- Re-run the notebook to use the improved model in the interface.